# 导入模块

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

In [38]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [39]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [40]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- AND tmer.shop_type !=2  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -31 day )
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -7 day )
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -1 day )
;
'''

df_order = query(sql1)
df_order.shape

执行定时任务：现在是每日的18点21分


Job "my_job1 (trigger: cron[hour='18', minute='21'], next run at: 2024-11-13 18:21:00 CST)" raised an exception
Traceback (most recent call last):
  File "g:\Virtualenv\Project\Lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_14272\1099452622.py", line 13, in my_job1
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
                 ^^^^^^^^^^^^
AttributeError: module 'datetime' has no attribute 'now'


(129651, 29)

# 合并数据

In [41]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(129651)

# 处理日期

In [42]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method'],
      dtype='object')

In [43]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df.shape

(129651, 31)

In [44]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                                                          59544
冰鉴火眸分<500命中强拒                                                                                             20959
命中强拒加入临时黑名单                                                                                                8381
年龄超过49岁或低于18岁                                                                                              4234
命中非免押用户拒绝                                                                                                  1984
                                                                                                          ...  
命中策略240703_14强拒, 命中自有模型强拒, 命中蚁盾分>50, 命中被执行人或曝光台                                                               1
命中自有模型强拒, 综合风险等级为3, 蚁盾分>=80, 评分等级D且综合风险等级为3,命中拒绝                                                              1
命中云商分低于496强拒, 命中策略240703_14强拒, 命中自有模型强拒, 命中融安分<700, 命中融安分低于680强拒(芝麻综合风险等级0豁免), 命中青云分<500, 评分等级低于D,命

# 处理备注信息

In [45]:
df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()

0         None
1         None
2         None
3         None
4         None
          ... 
129646    None
129647    None
129648    None
129649    None
129650    None
Name: total_describes, Length: 129651, dtype: object

In [46]:
df = df[df['sku_attributes'].notnull()]

In [47]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

def getfangan(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="租赁方案":
            return color_list[j]["value"]
df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)
df["租赁方案"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [48]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    124670
1      4968
Name: count, dtype: int64

# 渠道归属

In [49]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"    
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"], x['order_method']),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                              70048
搜索渠道                              30766
单人聊天会话中的小程序消息卡片（分享）                5054
八派信息                               4982
其他渠道场景渠道。                          4227
生活号                                2859
我的小程序入口                            2376
小程序商家消息（服务提醒）                      1966
九州信息                               1161
小舞直播                               1136
抖音渠道                                998
支付宝社群                               871
付费灯火                                724
未知渠道                                716
付费流量(通过商家数字推广平台，灯火等投放的广告)           425
支付宝 push 消息（同1014）                  417
付费邦道-首页                             203
支付成功页                               137
线下易拉宝                               105
邦道-自定义                              104
扫描二维码                                75
苹果SL23                               48
卡包\r\n                               43
支付宝客户端首页                             29
系统桌面图标                             

# 订单去重

In [50]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [51]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [52]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan and "待审核" not in e: 
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消    50281
前置拦截     21949
机审强拒     11668
无法联系      3613
客户取消      1635
人审拒绝       973
待审核         87
Name: count, dtype: int64

# 保留商家的数据

In [53]:
df_contain = df.copy()

# 剔除商家数据

In [54]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)

df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="小蚂蚁租机"].index,inplace=True)
df.drop(df[df['merchant_name']=="线下小店"].index,inplace=True)
df.drop(df[df['merchant_name'].str.contains(pat='探路者',regex=False)==True].index,inplace=True) 

# 去重订单明细导出

In [55]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-11-12'

# 导入服务订单

In [56]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(16406, 52)

In [57]:
df["下单时段"].drop_duplicates()

0         2024-10-12 00:
100       2024-10-12 01:
202       2024-10-12 02:
266       2024-10-12 03:
331       2024-10-12 04:
               ...      
128010    2024-11-12 14:
128317    2024-11-12 15:
128690    2024-11-12 16:
129048    2024-11-12 17:
129513    2024-11-12 18:
Name: 下单时段, Length: 763, dtype: object

In [58]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)
df = df[df['审核状态']!='已退款']
df['是否出库'] = np.where((df['审核状态']!='人审拒绝')&(df['是否出库']==1), 1, 0)


# iPhone 16 全系列

In [59]:

df_16 = df[df['product_name'].str.contains(r'iPhone 16|iPhone16')]
df_16 = df_16[df_16['product_name']!='iPhone 16 2024年新机 200元权益包']
df_16['product_name'] = df_16['product_name'].str.replace(' ', '', regex=False)
# df_16['型号内存'] = df_16['product_name'].str.split(' 国').str.get(0).str.split(' 2').str.get(0).str.replace(' ', '', regex=False).str.replace('首月5折', '', regex=False)
df_16['型号内存'] = df_16['product_name'].str.extract(r'(iPhone ?16(ProMax|Pro|Plus)?)', expand=True)[0]

def iPhone(df, qd_names):
    # df_16_group = df_16.groupby(['归属渠道','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_16_group = df.groupby([f'{qd_names}','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})

    df_16_group.rename(columns={'order_id':'去重订单数'},inplace=True )

    df_16_group["拦截率"] = df_16_group["前置拦截"]/df_16_group["去重订单数"]
    df_16_group["拦截率"] = df_16_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_16_group['预授权通过率'] = df_16_group["是否进件"]/df_16_group["去重订单数"]
    df_16_group["预授权通过率"] = df_16_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_16_group["强拒比例"] = df_16_group["机审强拒"]/df_16_group["是否进件"]
    df_16_group["强拒比例"] = df_16_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_16_group["机审通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]

    df_16_group["风控通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]-df_16_group["人审拒绝"]
    df_16_group["风控通过率"] = df_16_group["风控通过件"]/df_16_group["是否进件"]
    df_16_group["风控通过率"] = df_16_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_16_group["取消率"] = df_16_group["客户取消"]/df_16_group["是否进件"]
    df_16_group["取消率"] = df_16_group["取消率"].apply(lambda x:format(x,".2%"))


    df_16_group["人审拒绝率"]=df_16_group["人审拒绝"]/df_16_group["是否进件"]
    df_16_group["人审拒绝率"]=df_16_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_16_group["进件出库率"]=df_16_group["是否出库"]/df_16_group["是否进件"]
    df_16_group["进件出库率"]=df_16_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_16_group["订单出库率"]=df_16_group["是否出库"]/df_16_group["去重订单数"]
    df_16_group["订单出库率"]=df_16_group["订单出库率"].apply(lambda x:format(x,".2%"))

    df_16_group = df_16_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核"]]
    df_16_group
    return df_16_group


In [60]:
# 计算环比
def calculate_growth_rate(current, previous):
    if previous == 0:
        return float('inf') if current > 0 else float('-inf')
    return (current - previous) / previous

In [61]:
# 获取渠道的具体渠道
qd_names = ['来源渠道', '归属渠道']
indx1 = 1
qd_group = iPhone(df_16, qd_names[indx1])
qd_lists = ['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']
qd_list =  qd_group.reset_index()[qd_names[indx1]][qd_group.reset_index()[qd_names[indx1]].isin(qd_lists)].drop_duplicates().to_list()


In [62]:
qd_group

去重订单数  前置拦截      拦截率  是否进件  预授权通过率  机审强拒  \
归属渠道      型号内存           下单日期                                                   
九州信息      iPhone16       2024-10-22      1     0    0.00%     0   0.00%     0   
                         2024-10-25      1     1  100.00%     0   0.00%     0   
                         2024-10-26      1     1  100.00%     0   0.00%     0   
                         2024-10-27      2     1   50.00%     1  50.00%     0   
                         2024-10-28      1     1  100.00%     0   0.00%     0   
...                                    ...   ...      ...   ...     ...   ...   
邦道-自定义    iPhone16ProMax 2024-10-15      1     1  100.00%     0   0.00%     0   
                         2024-10-16      7     0    0.00%     3  42.86%     2   
                         2024-10-17      4     0    0.00%     1  25.00%     0   
                         2024-10-18      1     0    0.00%     0   0.00%     0   
首页十二宫格及更多 iPhone16ProMax 2024-10-12      1     0    0.00%     0   0.00%     0   

                                       强拒比例  机审通过件  人审拒绝  风控通过件    风控通过率  \
归属渠道      型号内存           下单日期                                              
九州信息      iPhone16       2024-10-22    nan%      0     0      0     nan%   
                         2024-10-25    nan%      0     0      0     nan%   
                         2024-10-26    nan%      0     0      0     nan%   
                         2024-10-27   0.00%      1     0      1  100.00%   
                         2024-10-28    nan%      0     0      0     nan%   
...                                     ...    ...   ...    ...      ...   
邦道-自定义    iPhone16ProMax 2024-10-15    nan%      0     0      0     nan%   
                         2024-10-16  66.67%      1     0      1   33.33%   
                         2024-10-17   0.00%      1     0      1  100.00%   
                         2024-10-18    nan%      0     0      0     nan%   
首页十二宫格及更多 iPhone16ProMax 2024-10-12    nan%      0     0      0     nan%   

                                     客户取消  无法联系  出库前风控强拒  待审核  
归属渠道      型号内存           下单日期                                  
九州信息      iPhone16       2024-10-22     0     0        0    0  
                         2024-10-25     0     0        0    0  
                         2024-10-26     0     0        0    0  
                         2024-10-27     1     0        0    0  
                         2024-10-28     0     0        0    0  
...                                   ...   ...      ...  ...  
邦道-自定义    iPhone16ProMax 2024-10-15     0     0        0    0  
                         2024-10-16     0     1        0    0  
                         2024-10-17     0     1        0    0  
                         2024-10-18     0     0        0    0  
首页十二宫格及更多 iPhone16ProMax 2024-10-12     0     0        0    0  

[1608 rows x 15 columns]

In [63]:
# 渠道明细 去重订单数
qd_type = ['去重订单数', '是否进件']
indx2 = 0
df_16_group_qcdd = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
df_16_group_qcdd = df_16_group_qcdd.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
df_pivot_qcdd = df_16_group_qcdd.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
df_pivot_qcdd = df_pivot_qcdd[ qd_list].reset_index()

In [64]:
# 渠道明细 是否进件
indx2 = 1
df_16_group_jj = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
df_16_group_jj = df_16_group_jj.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
df_pivot_jj = df_16_group_jj.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
df_pivot_jj = df_pivot_jj[ qd_list].reset_index()

In [65]:
# 渠道数据汇总
current = df_pivot_jj['下单日期'].iloc[-1].strftime('%m%d')
previous = df_pivot_jj['下单日期'].iloc[-2].strftime('%m%d')
df_16_qcdd = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
df_16_jj = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
for qd_name in qd_list:
    df_16_qcdd[qd_name] =  [df_pivot_qcdd[qd_name].diff().iloc[-1],  (df_pivot_qcdd[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]
    df_16_jj[qd_name] =  [df_pivot_jj[qd_name].diff().iloc[-1],  (df_pivot_jj[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]

indx = 0
df_pivot_qcdds = pd.concat([df_pivot_qcdd[indx:], df_16_qcdd])
df_pivot_jjs = pd.concat([df_pivot_jj[indx:], df_16_jj])
# df_pivot_jjs

# 设置定时任务

In [66]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

hour = 18
minute = 21
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1():
    print(f'执行定时任务：现在是每日的{hour}点{minute}分')
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\18.iPhone16预租数据/'
    # path1 = 'F:/需求/迪瓜租机/18.iPhone16预租数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl') as writer:
        df_pivot_qcdds.to_excel(writer, sheet_name='iPhone16系列去重订单数汇总', index=False)
    with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
        df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
        qd_group.to_excel(writer, sheet_name='iPhone16系列明细')
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的16点55分执行
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# 启动scheduler1
scheduler1.start()
    

In [67]:
# ip16 = pd.read_excel(r'\\digua\迪瓜租机\18.iPhone16预租数据/iPhone 16系列_2024092018.xlsx')
# ip16 = ip16.ffill()
# ip16['下单日期2'] = pd.to_datetime(ip16['下单日期']).dt.date
# ip16_group = ip16.groupby(['来源渠道', '下单日期2']).agg({'去重订单数': 'sum', '是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['来源渠道', '下单日期2'], ascending=(False, False))
# # ip16_group = ip16_group.sort_values(by=['下单日期2'], ascending=(False))
# # with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
# #         ip16_group.to_excel(writer, sheet_name='iPhone16系列转化')
# ip16_group

In [68]:

# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'去重订单数': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='去重订单数')
# df_pivot1 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
#         df_pivot1.to_excel(writer, sheet_name='iPhone16系列转化(去重订单数)')
# # ip16_group

In [69]:
# ip16['去重订单数'].sum()

In [70]:
# 
# ip16_group['去重订单数'].sum()

In [71]:
# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='是否进件')
# df_pivot2 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl', mode='a') as writer:
#         df_pivot2.to_excel(writer, sheet_name='iPhone16系列转化(进件)')

# 待审核

In [72]:
# df_dsh = df[df['待审核']==1]
# df_dsh = df_dsh[df_dsh['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_dsh = df_dsh[['order_number', 'create_time', 'product_name', '租赁方案', '内存', '颜色', '来源渠道']]
# df_dsh.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'product_name': '型号'}, inplace=True)
# df_dsh


# with pd.ExcelWriter('F:/需求/iPhone16.xlsx', engine='openpyxl') as writer:
#     df_dsh.to_excel(writer, sheet_name='待审核', index=False)